In [1]:
import pandas as pd
import numpy as np

In [29]:
data=pd.read_csv(r"C:\Users\LENOVO\Downloads\Ineuron\obesedata\notebook\data\ObesityDataSet.csv")
data.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,Female,21.0,1.62,64.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,0.0,1.0,no,Public_Transportation,Normal_Weight
1,Female,21.0,1.52,56.0,yes,no,3.0,3.0,Sometimes,yes,3.0,yes,3.0,0.0,Sometimes,Public_Transportation,Normal_Weight
2,Male,23.0,1.80,77.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,2.0,1.0,Frequently,Public_Transportation,Normal_Weight
3,Male,27.0,1.80,87.0,no,no,3.0,3.0,Sometimes,no,2.0,no,2.0,0.0,Frequently,Walking,Overweight_Level_I
4,Male,22.0,1.78,89.8,no,no,2.0,1.0,Sometimes,no,2.0,no,0.0,0.0,Sometimes,Public_Transportation,Overweight_Level_II


In [30]:
data['NObeyesdad'].unique()

array(['Normal_Weight', 'Overweight_Level_I', 'Overweight_Level_II',
       'Obesity_Type_I', 'Insufficient_Weight', 'Obesity_Type_II',
       'Obesity_Type_III'], dtype=object)

In [31]:
wt_map={'Normal_Weight':1, 'Overweight_Level_I':2, 'Overweight_Level_II':3,
       'Obesity_Type_I':4, 'Insufficient_Weight':0, 'Obesity_Type_II':5,
       'Obesity_Type_III':6}
data['NObeyesdad']=data['NObeyesdad'].map(wt_map)
data['NObeyesdad']

0       1
1       1
2       1
3       2
4       3
       ..
2106    6
2107    6
2108    6
2109    6
2110    6
Name: NObeyesdad, Length: 2111, dtype: int64

In [32]:
## Independent and dependent features
X = data.drop(labels=['NObeyesdad'],axis=1)
y = data[['NObeyesdad']]

In [33]:
categorical_cols = X.select_dtypes(include='object').columns
numerical_cols = X.select_dtypes(exclude='object').columns

In [34]:
categorical_cols

Index(['Gender', 'family_history_with_overweight', 'FAVC', 'CAEC', 'SMOKE',
       'SCC', 'CALC', 'MTRANS'],
      dtype='object')

In [35]:
numerical_cols

Index(['Age', 'Height', 'Weight', 'FCVC', 'NCP', 'CH2O', 'FAF', 'TUE'], dtype='object')

In [36]:
from sklearn.impute import SimpleImputer ## HAndling Missing Values
from sklearn.preprocessing import StandardScaler # HAndling Feature Scaling
from sklearn.preprocessing import OrdinalEncoder # Ordinal Encoding
from sklearn.preprocessing import OneHotEncoder,LabelEncoder # Onehot Encoding
## pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [37]:
## Numerical Pipeline
num_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='median')),
    ('scaler',StandardScaler(with_mean=False))

    ]

)

# Categorigal Pipeline
cat_pipeline = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        # ('ordinalencoder', OrdinalEncoder(categories=[Weather_c,Road_c,order_c,vehicle_c,Festival_c,City_c,])),
       ('onehotencoder', OneHotEncoder()),
        ('scaler', StandardScaler(with_mean=False))
    ] 
)

preprocessor=ColumnTransformer([
('num_pipeline',num_pipeline,numerical_cols),
('cat_pipeline',cat_pipeline,categorical_cols)
])


In [38]:
## Train test split

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.30,random_state=30)

In [39]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out()) ##we use traansform to avoid data leakage.



In [40]:
y_train.head()

,NObeyesdad
1211,4
427,1
1429,4
1686,5
1299,4


In [48]:
#Model Training
## Import the necessary libraries:-

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
# from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
# from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [47]:


def evaluate_model(true, predicted):
    report = classification_report(true, predicted)
    return report

# Train multiple models
models = {
    'DecisionTreeClassifier': DecisionTreeClassifier(),
    'LogisticRegressionClassifier': LogisticRegression(),
    'SVC': SVC()
}

model_list = []

for model_name, model in models.items():
    model.fit(X_train, y_train)

    # Make Predictions
    y_pred = model.predict(X_test)

    classification_matrix = evaluate_model(y_test, y_pred)

    print(model_name)
    model_list.append(model_name)

    print('Model Training Performance')
    print("Classification Report:", classification_matrix)

    print('=' * 35)
    print('\n')


DecisionTreeClassifier
Model Training Performance
Classification Report:               precision    recall  f1-score   support

           0       0.95      0.88      0.91        78
           1       0.76      0.89      0.82        82
           2       0.89      0.84      0.87        88
           3       0.91      0.89      0.90        84
           4       0.94      0.91      0.93       115
           5       0.94      0.98      0.96        96
           6       1.00      0.97      0.98        91

    accuracy                           0.91       634
   macro avg       0.91      0.91      0.91       634
weighted avg       0.92      0.91      0.91       634



LogisticRegressionClassifier
Model Training Performance
Classification Report:               precision    recall  f1-score   support

           0       0.85      0.90      0.88        78
           1       0.68      0.66      0.67        82
           2       0.74      0.69      0.72        88
           3       0.77      0.7